In [60]:
import pandas as pd
from bs4 import BeautifulSoup
import re

In [ ]:
#import s3fs
#df1 = pd.read_csv('s3://shirley001/chunk78k.csv') if dat is public from aws bucket

## Load data and check data format

In [2]:
df1 = pd.read_csv('78k.csv')
df2 = pd.read_csv('78_150k.csv')
df3 = pd.read_csv('150_240k.csv')
df4 = pd.read_csv('240_320k.csv')


//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dfs = pd.concat([df1,df2,df3,df4])

In [4]:
dfs.shape

(324059, 10)

In [5]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 324059 entries, 0 to 84122
Data columns (total 10 columns):
Id             324059 non-null int64
abstract       296782 non-null object
title          323870 non-null object
authors        323871 non-null object
journal        324059 non-null object
journal_abv    324059 non-null object
date           116545 non-null object
affiliation    323922 non-null object
records        323647 non-null object
keywords       37883 non-null object
dtypes: int64(1), object(9)
memory usage: 27.2+ MB


In [29]:
dfs['abstract'].iloc[2]


'<div class="abstr">\n <h3>\n  Abstract\n </h3>\n <div class="">\n  <p>\n   Patients suffering from cholestasis, the slowing or stoppage of bile flow, commonly report experiencing an intense, chronic itch. Numerous pruritogens are up-regulated in cholestatic patient sera, including bile acids (BAs). Acute injection of BAs results in itch in both mice and humans, and BA-modulating therapy is effective in controlling patient itch. Here, we present evidence that human sensory neuron-expressed Mas-related G protein-coupled receptor X4 (MRGPRX4), an orphan member of the\n   <i>\n    Mrgpr\n   </i>\n   family of GPCRs, is a BA receptor. Using Ca\n   <sup>\n    2+\n   </sup>\n   imaging, we determined that pathophysiologically relevant levels of numerous BAs activated MRGPRX4. No mouse Mrgpr orthologs were activated by BAs. To assess the in vivo relevance of BA activation of MRGPRX4, we generated a humanized mouse with targeted expression of MRGPRX4 in itch-encoding sensory neurons. BAs activ

In [7]:
dfs['title'].iloc[0]

'<h1>\n What we know and do not know about the cannabinoid receptor 2 (CB2).\n</h1>\n'

## Strip html tags and refill date to clean data

In [10]:
df = dfs.copy()

In [50]:
# remove html tags
def strip_html_tags(text):
    if type(text) != str:
        return 'NA'
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text.replace('\n','')

In [53]:
%%time
df.abstract = df.abstract.apply(lambda x: strip_html_tags(x))

CPU times: user 1min 37s, sys: 878 ms, total: 1min 38s
Wall time: 1min 38s


In [56]:
%%time
df.title = df.title.apply(lambda x: strip_html_tags(x))

CPU times: user 32.6 s, sys: 187 ms, total: 32.8 s
Wall time: 33.1 s


In [123]:
%%time

df.keywords = df.keywords.apply(lambda x: strip_html_tags(x))


CPU times: user 7.57 s, sys: 156 ms, total: 7.73 s
Wall time: 7.8 s


In [58]:
df.records.iloc[1]

'<div class="cit">\n <a abstractlink="yes" alsec="jour" alterm="Trends Pharmacol Sci." href="#" title="Trends in pharmacological sciences.">\n  Trends Pharmacol Sci.\n </a>\n 2006 Sep;27(9):475-81. Epub  2006 Aug 4.\n</div>\n'

In [73]:
type(df.date.iloc[1]),type(df.date.iloc[0])


(float, str)

In [75]:
# get date and year information
def get_date(record): # get date from record
    date = re.findall(r'</a>\n(.*)\n</div>',record)[0]
    return date

In [87]:
%%time
for i in range(211742,324059):
    if type(df.date.iloc[i]) != str and type(df.records.iloc[i]) == str:
        df.date.iloc[i] = get_date(df.records.iloc[i])

CPU times: user 1h 15min 33s, sys: 10.3 s, total: 1h 15min 43s
Wall time: 1h 16min 12s


## Check clean data and save 

In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 324059 entries, 0 to 84122
Data columns (total 10 columns):
Id             324059 non-null int64
abstract       324059 non-null object
title          324059 non-null object
authors        323871 non-null object
journal        324059 non-null object
journal_abv    324059 non-null object
date           323775 non-null object
affiliation    323922 non-null object
records        323647 non-null object
keywords       324059 non-null object
dtypes: int64(1), object(9)
memory usage: 37.2+ MB


In [134]:
df.iloc[:100000].to_csv('clean1.csv')

In [ ]:
df.iloc[100000:200000].to_csv('clean2.csv')


In [136]:
df.iloc[200000:].to_csv('clean3.csv')

In [137]:
df.head()

,Id,abstract,title,authors,journal,journal_abv,date,affiliation,records,keywords
0,24877594,Abstract It has been well appreciated tha...,What we know and do not know about the cannab...,"['Malfitano AM', 'Basu S', 'Maresz K', 'Bifulc...",Seminars in immunology.,Semin Immunol,2014 Oct;26(5):369-79.,"\n Dipartimento di Medicina e Chirurgia, Unive...","<div class=""cit"">\n <a abstractlink=""yes"" alse...",KEYWORDS: Cannabinoid receptor 2; Endocann...
1,16889837,Abstract Approximately 1% of the genome o...,Allosteric agonists of 7TM receptors: expandi...,"['Langmead CJ', 'Christopoulos A']",Trends in pharmacological sciences.,Trends Pharmacol Sci,2006 Sep;27(9):475-81. Epub 2006 Aug 4.,\n Psychiatry Centre of Excellence for Drug Di...,"<div class=""cit"">\n <a abstractlink=""yes"" alse...",NA
2,31068464,Abstract Patients suffering from cholesta...,MRGPRX4 is a G protein-coupled receptor activ...,"['Meixiong J', 'Vasavda C', 'Snyder SH', 'Dong...",Proceedings of the National Academy of Science...,Proc Natl Acad Sci U S A,2019 May 21;116(21):10525-10530.,\n Solomon H. Snyder Department of Neuroscienc...,"<div class=""cit"">\n <a abstractlink=""yes"" alse...",KEYWORDS: MRGPRX4; bile acids; cholestasis...
3,30610192,Abstract Alcohol use disorder (AUD) is a ...,"Modulation of Gpr39, a G-protein coupled rece...","['Cuzon Carlson VC', 'Ford MM', 'Carlson TL', ...",Neuropsychopharmacology : official publication...,Neuropsychopharmacology,2019 May;44(6):1103-1113.,"\n Division of Neuroscience, Oregon National P...","<div class=""cit"">\n <a abstractlink=""yes"" alse...",NA
4,30289386,Abstract Activation of heterotrimeric G p...,Simulation of spontaneous G protein activatio...,"['Sun X', 'Singh S', 'Blumer KJ', 'Bowman GR']",eLife.,Elife,2018 Oct 5;7. pii: e38465.,\n Department of Biochemistry and Molecular Bi...,"<div class=""cit"">\n <a abstractlink=""yes"" alse...",KEYWORDS: G proteins; Markov state models;...
